In [16]:
import pandas as pd

df = pd.read_csv("C:/Users/usame/Downloads/lemmatized_data.xls")
query_text = df.iloc[4]["text"]  

In [17]:
df_lem = pd.read_csv("tfidf_lemmatized.csv")
df_stem = pd.read_csv("tfidf_stemmed.csv")

In [21]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\usame\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [22]:
from sklearn.metrics.pairwise import cosine_similarity

query_vector = df_lem.iloc[4].values.reshape(1, -1)  # same index

In [23]:
similarities = cosine_similarity(query_vector, df_lem)[0]
top5_idx = similarities.argsort()[-6:-1][::-1]  # top 5 (excluding itself)
print(df.iloc[top5_idx])
print(similarities[top5_idx])

                                            text
6           harry potter collection harry potter
4415    harry potter chamber secret harry potter
2       harry potter chamber secret harry potter
0     harry potter halfblood prince harry potter
615   harry potter halfblood prince harry potter
[0.79398002 0.73513937 0.73513937 0.72475914 0.72475914]


In [26]:
from sklearn.metrics.pairwise import cosine_similarity

query_vector = df_stem.iloc[4].values.reshape(1, -1)  # same index

In [27]:
similarities = cosine_similarity(query_vector, df_stem)[0]
top5_idx = similarities.argsort()[-6:-1][::-1]  # top 5 (excluding itself)
print(df.iloc[top5_idx])
print(similarities[top5_idx])

                                              text
6             harry potter collection harry potter
10674  harry potter philosopher stone harry potter
4415      harry potter chamber secret harry potter
2         harry potter chamber secret harry potter
0       harry potter halfblood prince harry potter
[0.81941012 0.75758536 0.74704024 0.74704024 0.73649198]


In [28]:
similarities = cosine_similarity(query_vector, df_stem)[0]
top5_idx = similarities.argsort()[-6:-1][::-1]  # top 5 (excluding itself)
print(df.iloc[top5_idx])
print(similarities[top5_idx])

                                              text
6             harry potter collection harry potter
10674  harry potter philosopher stone harry potter
4415      harry potter chamber secret harry potter
2         harry potter chamber secret harry potter
0       harry potter halfblood prince harry potter
[0.81941012 0.75758536 0.74704024 0.74704024 0.73649198]


In [42]:
from nltk.tokenize import word_tokenize

tokens = word_tokenize(query_text.lower())

In [43]:
from gensim.models import Word2Vec
import numpy as np

model = Word2Vec.load("C:/Users/usame/Downloads/web2vec/word2vec_lemmatized_cbow_win2_dim100.model")

def get_avg_vector(tokens, model):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    if len(vectors) == 0:
        return np.zeros(model.vector_size)
    return np.mean(vectors, axis=0)
    
query_vec = get_avg_vector(tokens, model)

In [44]:
from gensim.models import Word2Vec
import numpy as np

model = Word2Vec.load("C:/Users/usame/Downloads/web2vec/word2vec_lemmatized_cbow_win4_dim100.model")

def get_avg_vector(tokens, model):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    if len(vectors) == 0:
        return np.zeros(model.vector_size)
    return np.mean(vectors, axis=0)
    
query_vec = get_avg_vector(tokens, model)

In [45]:
from gensim.models import Word2Vec
import numpy as np

model = Word2Vec.load("C:/Users/usame/Downloads/web2vec/word2vec_lemmatized_cbow_win4_dim300.model")

def get_avg_vector(tokens, model):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    if len(vectors) == 0:
        return np.zeros(model.vector_size)
    return np.mean(vectors, axis=0)
    
query_vec = get_avg_vector(tokens, model)

In [46]:
from gensim.models import Word2Vec
import numpy as np

model = Word2Vec.load("C:/Users/usame/Downloads/web2vec/word2vec_lemmatized_cbow_win2_dim300.model")

def get_avg_vector(tokens, model):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    if len(vectors) == 0:
        return np.zeros(model.vector_size)
    return np.mean(vectors, axis=0)
    
query_vec = get_avg_vector(tokens, model)

In [47]:
df["text"] = df["text"].fillna("").astype(str)

In [41]:
corpus_vecs = []
for text in df["text"]:
    t = word_tokenize(text.lower())
    corpus_vecs.append(get_avg_vector(t, model))

# Cosine similarity
sims = cosine_similarity([query_vec], corpus_vecs)[0]
top5 = sims.argsort()[-6:-1][::-1]

In [51]:
print("Query sentence:")
print(df.iloc[4]["text"]) 

print("\nTop 5 most similar sentences:")
for i in top5:
    print(f"- {df.iloc[i]['text']}")

Query sentence:
harry potter boxed set book harry potter

Top 5 most similar sentences:
- harry potter collection harry potter
- harry potter chamber secret harry potter
- harry potter chamber secret harry potter
- harry potter la piedra filosofal harry potter
- harry potter goblet fire harry potter


In [53]:
top5_sets = []

for i in top5:
    text = df.iloc[i]["text"]
    tokens = word_tokenize(str(text).lower())  
    top5_sets.append(set(tokens))  

In [54]:
print(top5_sets)

[{'potter', 'collection', 'harry'}, {'potter', 'secret', 'chamber', 'harry'}, {'potter', 'secret', 'chamber', 'harry'}, {'potter', 'piedra', 'filosofal', 'harry', 'la'}, {'potter', 'goblet', 'harry', 'fire'}]


In [56]:
query_text = df.iloc[4]["text"]
query_set = set(word_tokenize(str(query_text).lower()))

In [57]:
from nltk.tokenize import word_tokenize

query_text = df.iloc[4]["text"]  
query_tokens = set(word_tokenize(str(query_text).lower()))

def jaccard_similarity(set1, set2):
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    return len(intersection) / len(union) if union else 0

jaccard_scores = []
for s in top5_sets:
    score = jaccard_similarity(query_tokens, s)
    jaccard_scores.append(score)

import pandas as pd

matrix = pd.DataFrame(
    [jaccard_scores],
    index=["Query"],
    columns=[f"Top-{i+1}" for i in range(len(top5_sets))]
)

print(matrix)

          Top-1     Top-2     Top-3  Top-4     Top-5
Query  0.333333  0.285714  0.285714   0.25  0.285714
